In [6]:
import os
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt


PROJECT_ROOT = Path.cwd()


if (PROJECT_ROOT / "notebooks").name == "notebooks" and not (PROJECT_ROOT / "pyproject.toml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent


candidate_paths = [
    PROJECT_ROOT / "data" / "08_reporting" / "model_comparison.json",
    PROJECT_ROOT / "data" / "reporting" / "model_comparison.json",
]

MODEL_COMPARISON_PATH = None
for p in candidate_paths:
    if p.exists():
        MODEL_COMPARISON_PATH = p
        break

if MODEL_COMPARISON_PATH is None:
    raise FileNotFoundError(
        f"Nie znalazłem model_comparison.json.\n"
        f"Sprawdzone ścieżki:\n" + "\n".join(str(p) for p in candidate_paths)
    )

print(f"Używam pliku metryk: {MODEL_COMPARISON_PATH}")

with open(MODEL_COMPARISON_PATH, "r") as f:
    results = json.load(f)


df = pd.DataFrame(results["models"]).T
metrics = ["accuracy", "precision", "recall", "f1", "roc_auc"]
df_plot = df[metrics]

plt.figure(figsize=(10, 6))
df_plot.plot(kind="bar")
plt.title("Porównanie metryk modeli")
plt.xlabel("Model")
plt.ylabel("Wartość metryki")
plt.xticks(rotation=0)
plt.tight_layout()


plots_dir = PROJECT_ROOT / "docs" / "plots"
plots_dir.mkdir(parents=True, exist_ok=True)

out_path = plots_dir / "metrics_comparison.png"
plt.savefig(out_path)
plt.close()

print(f"Zapisano wykres: {out_path}")


Używam pliku metryk: E:\Projekty\ASI\ai-credit-scoring\data\08_reporting\model_comparison.json
Zapisano wykres: E:\Projekty\ASI\ai-credit-scoring\docs\plots\metrics_comparison.png


<Figure size 1000x600 with 0 Axes>

In [7]:
import os
import pickle
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt


PROJECT_ROOT = Path.cwd()
if (PROJECT_ROOT / "notebooks").name == "notebooks" and not (PROJECT_ROOT / "pyproject.toml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent


model_candidates = [
    PROJECT_ROOT / "data" / "06_models" / "custom_model.pkl",
    PROJECT_ROOT / "data" / "reporting" / "custom_model.pkl",
    PROJECT_ROOT / "data" / "06_models" / "best_model.pkl",
    PROJECT_ROOT / "data" / "reporting" / "best_model.pkl",
]

MODEL_PATH = None
for p in model_candidates:
    if p.exists():
        MODEL_PATH = p
        break

if MODEL_PATH is None:
    raise FileNotFoundError(
        "Nie znalazłem modelu custom/best.\n"
        "Sprawdzone ścieżki:\n" + "\n".join(str(p) for p in model_candidates)
    )

print(f"Używam modelu: {MODEL_PATH}")

with open(MODEL_PATH, "rb") as f:
    model = pickle.load(f)


train_candidates = [
    PROJECT_ROOT / "data" / "05_model_input" / "train.csv",
    PROJECT_ROOT / "data" / "model_input" / "train.csv",
]

TRAIN_PATH = None
for p in train_candidates:
    if p.exists():
        TRAIN_PATH = p
        break

if TRAIN_PATH is None:
    raise FileNotFoundError(
        "Nie znalazłem train.csv.\n"
        "Sprawdzone ścieżki:\n" + "\n".join(str(p) for p in train_candidates)
    )

print(f"Używam danych treningowych: {TRAIN_PATH}")

train = pd.read_csv(TRAIN_PATH)


TARGET_COL = "loan_status"

if TARGET_COL not in train.columns:
    raise KeyError(f"Kolumna targetu '{TARGET_COL}' nie istnieje w train.csv. Kolumny: {list(train.columns)}")

X = train.drop(columns=[TARGET_COL]).select_dtypes(include="number")

if not hasattr(model, "feature_importances_"):
    raise AttributeError(
        "Model nie ma atrybutu feature_importances_. Upewnij się, że to RandomForest/XGBoost/GBM itd."
    )

importances = model.feature_importances_
indices = importances.argsort()[::-1]


plt.figure(figsize=(10, 6))
plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), X.columns[indices], rotation=90)
plt.title("Ważność cech – Custom RandomForest")
plt.tight_layout()

plots_dir = PROJECT_ROOT / "docs" / "plots"
plots_dir.mkdir(parents=True, exist_ok=True)
out_path = plots_dir / "feature_importance.png"
plt.savefig(out_path)
plt.close()

print(f"Zapisano wykres: {out_path}")


Używam modelu: E:\Projekty\ASI\ai-credit-scoring\data\06_models\custom_model.pkl
Używam danych treningowych: E:\Projekty\ASI\ai-credit-scoring\data\05_model_input\train.csv
Zapisano wykres: E:\Projekty\ASI\ai-credit-scoring\docs\plots\feature_importance.png
